In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../input/amex-default-prediction/train_data.csv', nrows=120_000)
df.S_2 = pd.to_datetime(df.S_2)
df2 = pd.read_csv('../input/amex-default-prediction/train_labels.csv', nrows=120_000)
df = df.merge(df2,on='customer_ID',how='left')

In [ ]:
df.head()

# 1- Check target variable

In [ ]:
df2.sample(3)

customer_id is presented only, at least we don't see the time here, so the target is part of an state rather than a value of each step of the time series

In [ ]:
# Target is between 0 and 1
df2.target.value_counts().plot(kind ='bar', title = 'barplot for target variable')
plt.show()

We can see imbalanced data where 0 is represented by people who didn't fall to default and 1 ioc, a bit higher the proportion than I expected, but looks ok for me. 

# X variables

The problem is a time series problem as we can see, we have a client, a datetime and several variables to study. If we select the first client in the dataframe and select a variable at random (in this case we select D_41 (related to delinquency), we can plot it as follow: 

In [ ]:
fig, ax = plt.subplots(figsize =(12,5))
ax.plot(df[df.customer_ID == df.customer_ID[0]].S_2, df[df.customer_ID == df.customer_ID[0]].D_41, color ='r')
#ax.set_xlabel()
ax.set_title(f"var D_41 from customer {df.customer_ID[0]}")
plt.show()



### 2.a Delinquency Variables

In [ ]:
#Select only delinquency variables "D_"
df_delinquency = df.filter(like='D_')

In [ ]:
df_delinquency.describe()

In [ ]:
def plot_missing_points(df):
    fig, ax = plt.subplots(figsize = (20,5))
    sns.heatmap(df.isnull(), cbar = False ,linecolor = 'w')
    ax.set_title('Missing data points')
    plt.show()

    

In [ ]:
plot_missing_points(df_delinquency)

Let's check the distribution of each variable

In [ ]:
ax = df_delinquency.hist(layout = (47, 2),
                          alpha = 0.5, label ='x',
                          figsize = (30,100), bins =20,)

plt.show()

In [ ]:
df_delinquency.D_68.unique()

In general, we can see the distribution in delinquency is far from normal distribution and you can see some variables are representing ordinal  (or similar) encoding, for instance D_117 or D_68. Another point of view is to consider the distribution with periods. Chris Deotte built an interesting visualizations https://www.kaggle.com/code/cdeotte/time-series-eda

In [ ]:
def heatmap_corr(df):
    corr = df.corr()

    mask = np.triu(np.ones_like(corr, dtype = bool))
    f, ax = plt.subplots(figsize=(30,12), nrows =1, ncols=2)
    cmap = sns.diverging_palette(230,20, as_cmap = True)
    sns.heatmap(corr, mask = mask, cmap=cmap, square = True, vmax =.3, 
               center =0, linewidths = .5, ax=ax[0])
    ax[0].set_title('Correlation Heatmap')
    

    ax[1].hist(corr.unstack().values, bins =50)#.plot()
    ax[1].set_title('Correlation Distribution')
    plt.show()
    

    

In [ ]:
# Get a matrix correlation

heatmap_corr(df_delinquency)

In general the correlation is quite weak in most variables

In [ ]:
def boxplot_variables(df):
    #Boxplot for all D_ variables
    fig, ax = plt.subplots(figsize=(21,10))
    df.boxplot( rot=90)
    #ax.set_xticklabels(rotation = 45)
    ax.set_title('Boxplot Delinquency Variables')
    plt.show()
    


In [ ]:
#Boxplot for all D_ variables
boxplot_variables(df_delinquency)

few variables contain a much higher space like D_65 and D_71, then D_69 have highest variance

I will continue updating this notebook for EDA, expect this can help with some other ideas for this competition. 